## 네이버 스토어 리뷰 크롤링

네이버 가벽비교 사이트 [가격비교](https://search.shopping.naver.com/search) 에서 원하는 검색어를 통해 가격비교 접속하고 리뷰를 찾습니다.



In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import random
import pandas as pd

driver = webdriver.Chrome()
url = 'https://search.shopping.naver.com/search'
driver.get(url)

### 리뷰 콘텐츠 가져오기

먼저 쇼핑몰리뷰에서 리뷰를 가져오기위한 상위 요소를 찾아야 합니다.

<rigth><img src="https://drive.google.com/thumbnail?id=1MpRio3YTWNwpj7T20K9VsprTQxXd-oPl&sz=w1000" width="600"/></rigth>


#### 원하는 탭으로 제어 변환

드라이버에서 원하는 페이지를 접속하다 보면 새로운 탭으로 사이트가 열리는 경우가 생깁니다. 기본적으로 `driver`는 기존 탭에 머물러 있기때문에 새로 열린 탭으로 제어를 바꾸어 주어야 합니다.

In [ ]:
# 드라이버에 있는 탭 목록 반환
all_handles = driver.window_handles

# 두 번째 탭으로 전환
driver.switch_to.window(all_handles[1])

#### 상위 요소 목록 찾기

페이지의 css 소스는 항상 달라지므로 상시 확인이 필요합니다.

In [ ]:
# CSS_SELECTOR
cs = 'ul.reviewItems_list_review__deAmE li'
reviews = driver.find_elements(By.CSS_SELECTOR, cs)
print(len(reviews))

20


> CSS_SELECTOR를 띄어쓰기로 연속하여 입력하면 순서대로 상위요소 부터 접근

In [ ]:
# XPATH
xp = '//*[@id="section_review"]/ul/li'
reviews = driver.find_elements(By.XPATH, '//*[@id="section_review"]/ul/li')
print(len(reviews))


20


#### 하위 요소 내용 가져오기

In [ ]:
def get_content():
    reviews = driver.find_elements(By.XPATH, '//*[@id="section_review"]/ul/li')
    craw = []
    for rv in reviews:
        content = {}
        info = rv.find_elements(By.CSS_SELECTOR, 'span.reviewItems_etc__qcKuJ')
        content['id'] = info[1].get_attribute('innerText')
        content['date'] = info[2].get_attribute('innerText')
        content['text'] = rv.find_element(By.CSS_SELECTOR,
                                          'p.reviewItems_text___PQZh').get_attribute('innerText')
        content['rat'] = rv.find_element(By.CSS_SELECTOR,
                                         'span.reviewItems_star__7pG2_').get_attribute('innerText')
        content['img'] = rv.find_element(By.CSS_SELECTOR,
                                         'span.reviewItems_img_box__Mg_Ur img').get_attribute('src')
        craw.append(content)
    return craw

get_content()

[{'id': 'euge****',
  'date': '22.09.16.',
  'text': '새부리 KF94 마스크를 검색하니 네이버 랭킹 상위에 뜨고 가격도 착해서 호기심에 주문해봤어요\n특히 핑크베이지 색상이 예뻐보여서요\n무난하게 맨날 화이트 블랙만 사보고 핑크 계열은 안사봐서 궁금하기도 하고 무엇보다 가격이 메리트가 커서 ㅋㅋ\n장바구니에 담을 때 분명 9,900원이었는데 결제하려고 보니까 12,900원 뜨더라구요? 그래서 주문 못하고 있다가 하루에도 라이브를 몇 번 씩 하시고 그 타이밍에 주문하면 9,900원이라 운좋게 아침 8시반 라방하기 전 8시에 주문 성공해서 당일 출고해서 담날 바로 받았어요\n봄 여름 핑크가 아니라 딱 가을 겨울 톤다운 핑크예요~\n살구핑크베이지랄까? 차분한 톤이라 오히려 맘에 드네요\n다른 후기처럼 중형인데 일반 대형 사이즈라길래 중형으로 주문했더니 역시나 넉넉하게 잘 맞아요\n남편이 써도 잘 맞더라구요\n웬만한 보통 얼굴 크기 남성이라면 중형도 OK일듯!\n가로폭이 커서 얼굴 전체를 잘 감싸주고 세로는 약간 짧은듯한데 그렇다고 부족하진 않아요\n사진에 미밋 대형, 아이랑 어른용과 비교 사이즈 남겼으니 참고 부탁드려요\n일단 25매 벌크 포장 뜯었을 때는 딱히 불량이 안보이고 괜찮은데 제발 한달 사용 후기로 평범 1점 주는 일이 없기를 바래야겠어요\n암튼 가성비 마스크로 추천!',
  'rat': '평점5'},
 {'id': 'gemm****',
  'date': '22.05.03.',
  'text': '컬러 KF94 마스크를 너무나도 기다렸어요\n예쁜 컬러에 벌크포장과 저렴하면서 퀄리티 있는걸 내심 원했는데 제가 생각했던거와 100%일치🤭\n동호님 런칭 방송때 신상 스포하는줄 알았다가 배너뜬거 보고 묻지도 따지지도 않고 오픈런 구매했어요🤗\n포장 따위가 이렇게 예쁠일인가요\U0001f979\n에코브리즈 무광도 예뻤는데 유광도 반짝반짝✨넘 예뻐요\n50매 구성 나중에 만들어주셔서 좀 아쉽…라이트실버와 50매로 구매했으면

> find_elements 결과에서 순서를 활용하여 요소를 찾는 방법도 유용

### 페이지 변경하여 리뷰 노출
- url 링크에 페이지 번호가 노출되면 문자열 포맷으로 `get()` 함수를 반복 호출 가능 (쉬움)
- url 링크에 페이지 번호가 없으면 직접 버튼을 누르게 소스코드를 구현 (어려움)

<rigth><img src="https://drive.google.com/thumbnail?id=1-_rxsmlczs6aE_zwqjFvQeRFTOqUcoE9&sz=w1000" width="500"/></rigth>


#### 번호 목록 요소의 구성 파악

In [ ]:
pag = driver.find_elements(By.XPATH, '//*[@id="section_review"]/div[3]/a')
print(len(pag))
pag[10].get_attribute('innerText')

11


'다음'

> 처음 페이지 번호 요소를 가져온 경우 11개의 번호를 가져오며 마지막 번호의 내용은 '다음'

In [ ]:
pag[10].click()
sleep(1)
pag = driver.find_elements(By.XPATH, '//*[@id="section_review"]/div[3]/a')
print(len(pag))
pag[11].get_attribute('innerText')

12


'다음'

> '다음' 버튼을 클릭한 다음 바뀐 번호 목록을 가져오면 12개의 목록을 가져옴(이전이 추가됨)   

#### 반복문을 통해 번호 클릭하여 수집

In [ ]:

i = 1
result = []
while True:
    try:
        pag = driver.find_element(By.XPATH, f'//*[@id="section_review"]/div[3]/a[{i}]')
        pt = pag.get_attribute('innerText')
        print(pt)
        pag.click()
        if pt == '다음':
            i = 2
        else:
            i += 1
        sleep(1)
        result.extend(get_content())
    except:
        break


현재 페이지
1
2
3
4
5
6
7
8
9
10
다음
현재 페이지
11
12
13
14
15
16
17
18
19
20
다음
현재 페이지
21
22
23
24
25
26
27
28
29
30
다음
현재 페이지
31
32
33
34
35
36
37
38
39
40
다음
현재 페이지
41
42
43
44
45
46
47
48
49
50
다음
현재 페이지
51
52
53
54
55
56
57
58
59
60
다음
현재 페이지
61
62
63
64
65
66
67
68
69
70
다음
현재 페이지
71
72
73
74
75
76
77
78
79
80
다음
현재 페이지
81
82
83
84
85
86
87
88
89
90
다음
현재 페이지
91
92
93
94
95
96
97
98
99
100
다음


> 클릭한 버튼의 내용을 가져와 검사하여 다음 클릭할 순서를 선정하는 방식 (내용으로 구분가능한 경우)     
> 해당 방법 외에도 단순 순서를 고려한 방법과 목록 개수를 고려하여 조건문을 구성하고 클릭하는 방법등이 사용됨   

In [ ]:
df = pd.DataFrame(result)
df

,id,date,text,rat
0,euge****,22.09.16.,새부리 KF94 마스크를 검색하니 네이버 랭킹 상위에 뜨고 가격도 착해서 호기심에 ...,평점5
1,gemm****,22.05.03.,컬러 KF94 마스크를 너무나도 기다렸어요\n예쁜 컬러에 벌크포장과 저렴하면서 퀄리...,평점5
2,gemm****,22.05.06.,컬러 KF94 마스크를 너무나도 기다렸어요\n세상 어디에도 없는 라이트실버 H워월V...,평점5
3,jina****,22.07.22.,라이브 보고 방송하는언니 쓴거 넘 예뻐서 100개 주문했는데 가로사이즈가 저한테 좀...,평점5
4,pim4****,22.10.21.,고민고민하다 디럭스 실버를 주문했는데\n진짜 이쁩니다. 과하지 않지만 평범하지도 않...,평점5
...,...,...,...,...
915,yooj****,22.07.12.,색깔 이뻐요 넘 화사해 보이고. 여름 자외선에 기미잡티 안생기고 싶어서 옆에 좀더 ...,평점5
916,somi****,22.07.06.,엄마께서 친구분꺼 써보시고 맘에들어하셔서 주문했어요\n귀에 고무줄 부분이 쪼끔 땡기...,평점4
917,babu****,22.11.01.,"우리집 마스크는 비움입니다. 와이프도 비움을 사용하고, 저도 비움을 사용합니다. 가...",평점5
918,iny3****,22.11.15.,중형했더니 사이즈맞아요.\n코쪽은 살짝뜨는감이있어서 얼굴에 밀착시켜서 사용하면될것같...,평점5


In [ ]:
df.to_csv('./data/craw.csv', index=False)

### 이미지 추가 수집
이미지 태그인 `<img>` 내부의 `src` 요소에 있는 url 값을 저장했다면 해당 주소를 통해 이미지를 다운로드 할 수있습니다.

**urllib 모듈 활용**
urllib 라이브러리를 활용하면 url 주소로 부터 연결된 데이터를 컴퓨터에 다운로드 가능합니다.

- `urllib.request.urlretrieve(url,file)` : `url` 경로에있는 데이터를 `file`에 다운로드

In [ ]:
import urllib.request as urq
import os

# 저장 폴더 생성
os.makedirs('./image', exist_ok=True)

i = 0
for link in df['img']:
    try:
        # 이미지 링크(link)를 사용하여 이미지를 다운로드하고, 지정된 경로에 저장
        urq.urlretrieve(link, f'image/img_{i}.jpg')
    except Exception as e:
        print(f'{i}번째 사진 에러: {e}')
    i += 1

# 실습 해보기


#### [실습1]

네이버 검색창의 [뉴스](https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=)또는 [블로그](https://search.naver.com/search.naver?ssc=tab.blog.all&sm=tab_jum&query=)에서 크롤링 하기

1. 위의 검색에서 원하는 키워드로 검색하여 나온 요약 내용을 드래그를 통해 100개 이상 크롤링
    * 뉴스 게시물에 내용에는 제목, 요약글, 날짜, 언론사 포함
    * 블로그 게시물에 내용에는 제목, 요약글, 날짜, 블로그작성자 포함


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import random
import pandas as pd

driver = webdriver.Chrome()
url= "https://search.naver.com/search.naver?sm=tab_hty.top&where=news&ssc=tab.news.all&query=gpt&oquery=&tqi=jM8qswqo1LVssi4edCossssstdV-308164&ackey=ur3kcps3"
driver.get(url)


In [ ]:
xp = "/html"
news_box = driver.find_element(By.XPATH, xp)

for i in range(10):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', news_box )
    sleep(random.randint(2, 3)) # 2~3초 쉬었다가 반복


In [ ]:
//*[@id="fdr-dfb7e098f0394e62b45ceb9e0122d671"]/div/div/div/div[1]/div[2]

In [7]:
texts = driver.find_elements(By.CSS_SELECTOR, 'div._slog_visible')
texts

[<selenium.webdriver.remote.webelement.WebElement (session="b198a4832fb542695ab7a598beaf8f39", element="f.3E03057C5B59E4F475DE25623876D7E1.d.F775717406BE1E9912418020A1981A57.e.6413")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b198a4832fb542695ab7a598beaf8f39", element="f.3E03057C5B59E4F475DE25623876D7E1.d.F775717406BE1E9912418020A1981A57.e.9181")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b198a4832fb542695ab7a598beaf8f39", element="f.3E03057C5B59E4F475DE25623876D7E1.d.F775717406BE1E9912418020A1981A57.e.9888")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b198a4832fb542695ab7a598beaf8f39", element="f.3E03057C5B59E4F475DE25623876D7E1.d.F775717406BE1E9912418020A1981A57.e.10589")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b198a4832fb542695ab7a598beaf8f39", element="f.3E03057C5B59E4F475DE25623876D7E1.d.F775717406BE1E9912418020A1981A57.e.11385")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b198a4832fb54

In [8]:
for text in texts:
    print(text.get_attribute('innerText'))

뉴시스
5시간 전
네이버뉴스
올트먼 "챗GPT, 12월부터 성인용 콘텐츠 허용"
샘 올트먼 오픈AI 최고경영자(CEO)는 14일(현지 시간) 챗GPT가 올해 12월 성인 이용자를 대상으로 성인용 콘텐츠를 허용할 것이라고 밝혔다. 올트먼은 이날 소셜미디어 엑스(X)에 올린 글에서 "몇 주 안에, 사용자들이 GPT-4o에서 좋아했던 인간적인 특성을 더 잘 구현하는 새로운 버전의 챗GPT를 출시할 계획"이라며...
챗GPT, 성인에게 성적 콘텐츠 허용한다
조선일보
3시간 전
네이버뉴스
‘성인용 챗GPT’ 나온다…연령 인증하면 성적 대화 허용
동아일보
6시간 전
네이버뉴스
"챗GPT 야해진다"…12월부터 신청 성인에 성적 콘텐츠 허용
뉴스1
5시간 전
네이버뉴스
챗GPT ‘19금 대화’ 가능… “12월부터 성인만”
서울신문
6시간 전
네이버뉴스
관련뉴스 43건 전체보기
연합뉴스
10시간 전
네이버뉴스
세일즈포스-오픈AI 협업 확대…업무용 '슬랙'에 챗GPT 통합
이에 따라 세일즈포스의 서비스를 이용하는 기업들은 오픈AI의 챗GPT 내에서 '에이전트포스 360' 플랫폼에 접근할 수 있게 된다. 챗GPT에 질문이나 요청을 입력하면 영업 기록 조회나 고객과의 대화 검토, 도표로 시각화 등 여러 작업을 간편하게 할 수 있다고 회사 측은 설명했다. 또 업무용으로 많이 쓰이는...
세일즈포스, 오픈AI와 협력…"챗GPT 통해 기업용 바이브코딩 도입"
한국경제
8시간 전
네이버뉴스
"챗GPT서 잠재 고객 만나"…세일즈포스, 오픈AI·앤트로픽 협력 확대
지디넷코리아
5시간 전
네이버뉴스
세일즈포스, 기업용 AI 서비스 출시…오픈AI 챗GPT와 연동
전자신문
6시간 전
네이버뉴스
"기업용 AI 새 판 짠다"…세일즈포스·오픈AI 협업 확대
뉴스1
3시간 전
네이버뉴스
전자신문
6시간 전
네이버뉴스
오픈AI, 챗GPT서 성인 이용자 성적 대화 허용…월마트 상품도 구매
챗GPT'에 전자상거래 기능과 성인 대화 허용 정책을 동시에 확대하며 플랫폼의 기능과 이용 범위를 대폭 확장한다. 생성형

#### [실습2]

[네이버블로그](https://section.blog.naver.com/BlogHome.naver?directoryNo=0&currentPage=1&groupId=0)에서 크롤링 하기

1. 위의 블로그 홈에서 원하는 키워드로 검색하여 나온 게시물 내용을 10페이지 이상 크롤링
    * 게시물에 내용에는 제목, 요약글, 날짜, 작성자, 이미지 링크(url)가 포함
2. [심화] 이미지 링크를 활용하여 첫번째 썸네일 이미지를 저장
3. [심화] 날짜 검색을 활용하여 기간을 월별로 나누어 최근 6개월의 글을 크롤링

#### [참고] 날짜 만들기

In [ ]:
dateforms = []

daterng = pd.date_range(start='1/1/2020', end = '6/1/2022', freq='MS').strftime('%Y%m%d')

for i in range(len(daterng) - 1):
    dateforms.append(f'{daterng[i]}to{daterng[i+1]}')
dateforms

['20200101to20200201',
 '20200201to20200301',
 '20200301to20200401',
 '20200401to20200501',
 '20200501to20200601',
 '20200601to20200701',
 '20200701to20200801',
 '20200801to20200901',
 '20200901to20201001',
 '20201001to20201101',
 '20201101to20201201',
 '20201201to20210101',
 '20210101to20210201',
 '20210201to20210301',
 '20210301to20210401',
 '20210401to20210501',
 '20210501to20210601',
 '20210601to20210701',
 '20210701to20210801',
 '20210801to20210901',
 '20210901to20211001',
 '20211001to20211101',
 '20211101to20211201',
 '20211201to20220101',
 '20220101to20220201',
 '20220201to20220301',
 '20220301to20220401',
 '20220401to20220501',
 '20220501to20220601']

In [ ]:
# 현재 열려 있는 모든 탭(창)의 핸들을 가져옵니다.
all_handles = driver.window_handles

# 두 번째 탭으로 전환합니다. (인덱스는 0부터 시작하므로 두 번째 탭은 인덱스 1입니다.)
driver.switch_to.window(all_handles[1])

# 두 번째 탭에서 요소를 찾습니다. (예시: 페이지 제목 출력)
print(driver.title)

# 이제 두 번째 탭에서 find_element 또는 find_elements를 사용하여 원하는 요소를 찾을 수 있습니다.
# element = driver.find_element(By.CSS_SELECTOR, '원하는 CSS 셀렉터')
# elements = driver.find_elements(By.XPATH, '원하는 XPATH')

# 작업을 마친 후 원래 탭으로 돌아가려면 다음 코드를 사용합니다.
# driver.switch_to.window(all_handles[0])